# 電力交易市場數據分析課程 2024/05/31

In [6]:
import pandas as pd
import numpy as np

## 電力週報案例一：市場價格趨勢

In [7]:
# colab
url = 'https://raw.githubusercontent.com/godspeed833/course_demo_files/master/vpp_tbl_etp_historical_settlement_trading_colab.csv'
df_etp = pd.read_csv(url)

In [18]:
df_etp

,tranDate,tranHour,regBidQse,regPrice,srBidQse,srPrice,supBidQse,supPrice,datetime,month
0,2021-07-01,00:00:00,NaN,108.0,NaN,150.0,NaN,300.0,2021-07-01 00:00:00,2021-07
1,2021-07-01,01:00:00,NaN,150.0,NaN,150.0,NaN,300.0,2021-07-01 01:00:00,2021-07
2,2021-07-01,02:00:00,NaN,108.0,NaN,150.0,NaN,300.0,2021-07-01 02:00:00,2021-07
3,2021-07-01,03:00:00,NaN,108.0,NaN,150.0,NaN,300.0,2021-07-01 03:00:00,2021-07
4,2021-07-01,04:00:00,NaN,108.0,NaN,150.0,NaN,300.0,2021-07-01 04:00:00,2021-07
...,...,...,...,...,...,...,...,...,...,...
19627,2023-09-26,19:00:00,366.9,500.0,66.2,280.0,76.9,320.0,2023-09-26 19:00:00,2023-09
19628,2023-09-26,20:00:00,366.9,500.0,67.4,280.0,83.8,320.0,2023-09-26 20:00:00,2023-09
19629,2023-09-26,21:00:00,371.9,480.0,65.0,249.0,94.1,320.0,2023-09-26 21:00:00,2023-09
19630,2023-09-26,22:00:00,371.9,450.0,74.9,280.0,104.0,245.0,2023-09-26 22:00:00,2023-09


In [19]:
df_etp['datetime'] = df_etp['tranDate'] + ' ' + df_etp['tranHour']
df_etp['datetime'] = pd.to_datetime(df_etp['datetime'])
df_etp['month'] = df_etp.datetime.dt.to_period('M').astype(str)

In [20]:
df_etp.columns

Index(['tranDate', 'tranHour', 'regBidQse', 'regPrice', 'srBidQse', 'srPrice',
       'supBidQse', 'supPrice', 'datetime', 'month'],
      dtype='object')

In [21]:
#df_etp2 = df_etp[df_etp['datetime'] > '2022-01-01'][['datetime', 'regBidQse', 'regPrice', 'srBidQse', 'srPrice', 'supBidQse', 'supPrice']]
df_etp2 = df_etp[df_etp['datetime'] > '2022-01-01'][['datetime', 'month', 'regPrice', 'srPrice', 'supPrice']]

In [22]:
df_etp2.head()

,datetime,month,regPrice,srPrice,supPrice
4417,2022-01-01 01:00:00,2022-01,595.0,400.0,344.0
4418,2022-01-01 02:00:00,2022-01,595.0,400.0,344.0
4419,2022-01-01 03:00:00,2022-01,596.0,400.0,344.0
4420,2022-01-01 04:00:00,2022-01,596.0,400.0,344.0
4421,2022-01-01 05:00:00,2022-01,595.0,400.0,344.0


In [23]:
df_etp2.regPrice

4417     595.0
4418     595.0
4419     596.0
4420     596.0
4421     595.0
         ...  
19627    500.0
19628    500.0
19629    480.0
19630    450.0
19631    400.0
Name: regPrice, Length: 15215, dtype: float64

In [24]:
df_etp2.regPrice.describe()

count    15215.000000
mean       558.807852
std         77.395137
min        305.000000
25%        595.000000
50%        599.000000
75%        599.000000
max        600.000000
Name: regPrice, dtype: float64

In [25]:
df_etp2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15215 entries, 4417 to 19631
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  15215 non-null  datetime64[ns]
 1   month     15215 non-null  object        
 2   regPrice  15215 non-null  float64       
 3   srPrice   15214 non-null  float64       
 4   supPrice  15215 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 1.2+ MB


In [26]:
df_etp2.isnull().sum()

datetime    0
month       0
regPrice    0
srPrice     1
supPrice    0
dtype: int64

In [27]:
# 缺值視沒有交易，所以不補值
df_etp2[df_etp2.srPrice.isnull()]

,datetime,month,regPrice,srPrice,supPrice
4698,2022-01-12 18:00:00,2022-01,595.0,NaN,350.0


In [28]:
df_etp2.groupby(df_etp2.datetime.dt.to_period('M')).agg({'regPrice':'mean'})

,regPrice
datetime,
2022-01,596.726783
2022-02,598.407738
2022-03,599.270161
2022-04,597.979167
2022-05,597.295699
2022-06,598.195833
2022-07,599.041667
2022-08,599.271505
2022-09,598.791667


In [29]:
df_etp2.groupby([df_etp2.month, df_etp2.datetime.dt.hour]).agg({'regPrice':'mean','srPrice':'mean','supPrice':'mean'})

regPrice     srPrice    supPrice
month   datetime                                    
2022-01 0         596.766667  399.333333  346.000000
        1         596.903226  398.870968  345.548387
        2         597.193548  399.516129  345.548387
        3         597.161290  399.193548  345.161290
        4         597.225806  399.677419  344.709677
...                      ...         ...         ...
2023-09 19        493.769231  274.038462  287.500000
        20        491.653846  274.038462  283.461538
        21        449.846154  265.692308  270.576923
        22        415.384615  265.369231  266.923077
        23        373.869231  265.692308  253.653846

[504 rows x 3 columns]

In [30]:
df_etp3 = df_etp2.groupby([df_etp2.month,df_etp2.datetime.dt.hour]).agg({'regPrice':'mean','srPrice':'mean','supPrice':'mean',}).reset_index()
df_etp3.round().head()

,month,datetime,regPrice,srPrice,supPrice
0,2022-01,0,597.0,399.0,346.0
1,2022-01,1,597.0,399.0,346.0
2,2022-01,2,597.0,400.0,346.0
3,2022-01,3,597.0,399.0,345.0
4,2022-01,4,597.0,400.0,345.0


In [31]:
# pivot_table: 一個值，以row跟columns的方式展開
df_etp3.pivot_table(index='month', columns='datetime', values='regPrice')

datetime,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
month,,,,,,,,,,,,,,,,,,,,,
2022-01,596.766667,596.903226,597.193548,597.161290,597.225806,597.225806,596.677419,596.709677,596.774194,596.838710,...,596.806452,596.709677,596.387097,596.193548,596.161290,596.161290,596.258065,596.258065,596.354839,596.612903
2022-02,598.892857,598.928571,598.892857,598.785714,598.892857,598.892857,598.857143,598.821429,598.678571,598.357143,...,598.750000,598.928571,598.285714,597.464286,597.678571,597.250000,597.071429,597.464286,598.035714,598.035714
2022-03,599.419355,599.419355,599.387097,599.419355,599.354839,599.451613,599.387097,599.419355,599.290323,599.225806,...,599.322581,599.225806,599.161290,598.967742,598.741935,598.870968,598.838710,599.387097,599.354839,599.419355
2022-04,598.200000,598.166667,598.100000,598.100000,598.100000,598.200000,598.200000,598.133333,598.166667,598.100000,...,597.766667,597.466667,597.666667,597.866667,597.566667,597.500000,597.666667,597.600000,598.133333,598.233333
2022-05,597.322581,597.387097,597.322581,597.225806,597.161290,597.290323,597.387097,597.354839,597.483871,597.451613,...,597.451613,597.096774,596.838710,596.967742,596.451613,596.225806,597.161290,597.419355,597.870968,597.935484
2022-06,597.933333,597.933333,597.933333,597.933333,597.933333,597.933333,597.933333,597.933333,597.933333,597.300000,...,598.266667,598.266667,598.300000,598.433333,598.100000,598.400000,598.433333,598.666667,598.800000,598.800000
2022-07,599.032258,599.032258,599.032258,599.032258,599.064516,599.032258,599.032258,599.032258,599.032258,598.741935,...,598.967742,598.903226,599.193548,599.161290,599.193548,599.193548,599.161290,599.161290,599.129032,599.096774
2022-08,599.000000,599.000000,599.000000,599.000000,599.000000,599.000000,599.000000,599.000000,599.096774,599.096774,...,599.483871,599.483871,599.483871,599.483871,599.612903,599.645161,599.645161,599.612903,599.548387,599.096774
2022-09,598.800000,598.800000,598.800000,598.800000,598.800000,598.800000,598.800000,598.800000,598.800000,598.733333,...,598.800000,598.800000,598.800000,598.800000,598.800000,598.800000,598.800000,598.800000,598.800000,598.800000


In [32]:
import plotly.graph_objects as go

In [33]:
# Create traces
fig = go.Figure()

for month in df_etp3.month.unique():
    tmp = df_etp3[df_etp3.month == month]
    fig.add_trace(go.Scatter(x=tmp.datetime, y=tmp.regPrice,
                    mode='lines',
                    name=month))

fig.show()

In [34]:
import matplotlib.pyplot as plt

# 創建時間順序的顏色漸變並反轉顏色比例
unique_months = df_etp3['month'].unique()
color_scale = plt.cm.viridis(np.linspace(0, 1, len(unique_months)))

# 創建具有顏色漸變的 traces
fig = go.Figure()

for idx, month in enumerate(unique_months):
    tmp = df_etp3[df_etp3['month'] == month]
    color = f'rgba({color_scale[idx][0]*255},{color_scale[idx][1]*255},{color_scale[idx][2]*255},1)'
    fig.add_trace(go.Scatter(x=tmp['datetime'], y=tmp['regPrice'],
                             mode='lines',
                             name=month,
                             line=dict(color=color)))

fig.update_layout(
    title="調頻價格隨時間的變化",
    xaxis_title="24小時",
    yaxis_title="調頻價格",
    legend_title="月份"
)

fig.show()


In [35]:
import plotly.express as px
import pandas as pd

# 假设 df_etp3 包含了你的数据
# 可以透過类似以下的方式转换数据，以适应 Plotly Express 的格式要求
# df = pd.DataFrame({
#     'datetime': df_etp3['datetime'],
#     'regPrice': df_etp3['regPrice'],
#     'month': df_etp3['month']
# })

# 创建动画
fig = px.line(df_etp3, x='datetime', y='regPrice', animation_frame='month',
              title='Monthly regPrice Change', range_y=[0, 600])

# 显示动画
fig.show()


### 我將上面的程式碼貼給Chatgpt，問他如何產生dash介面
![image.png](attachment:image.png)

In [36]:
!pip install jupyter-dash -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.9 MB/s eta 0:00:00


In [37]:
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


# 创建 Dash 应用
# app = dash.Dash(__name__)
app = JupyterDash(__name__)

# 布局
app.layout = html.Div([
    dcc.Checklist(
        id='price-selector',
        options=[
            {'label': 'regPrice', 'value': 'regPrice'},
            {'label': 'suPrice', 'value': 'srPrice'},
            {'label': 'spPrice', 'value': 'supPrice'}
        ],
        value=['regPrice'],  # 默认选中 'regPrice'
    ),
    dcc.Graph(id='price-graph')
])

# 回调函数，更新图表
@app.callback(
    Output('price-graph', 'figure'),
    [Input('price-selector', 'value')]
)
def update_graph(selected_prices):
    # 创建动画
    fig = px.line(df_etp3, x='datetime', y=selected_prices, animation_frame='month',
                  title='Monthly Price Change', range_y=[0, 600])
    return fig

# 在colab運行時，不需要指定mode，不然會跑出兩個
app.run_server(port=8035)

/usr/local/lib/python3.10/dist-packages/dash/dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

Dash app running on:


<IPython.core.display.Javascript object>

## 電力週報案例二 機組發電資料分析

In [38]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd

# 定義數據
data = {
    'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00'],
    '太陽能': [23.7, 23.7, 23.7, 23.7, 23.7, 23.7, 23.7, 23.7],
    '其他再生能源': [2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1],
    '儲能': [0, 0, 0, 0, 0, 0, 0, 0],
    '風力': [23.5, 23.5, 23.5, 23.5, 23.5, 23.5, 23.5, 23.5],
    '水力': [20.2, 20.2, 20.2, 20.2, 20.2, 20.2, 20.2, 20.2],
    '抽蓄': [3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6],
    '燃油': [16.2, 16.2, 16.2, 16.2, 16.2, 16.2, 16.2, 16.2],
    '民營電廠-燃氣': [225.6, 225.6, 225.6, 225.6, 225.6, 225.6, 225.6, 225.6],
    '燃氣': [109.5, 109.5, 109.5, 109.5, 109.5, 109.5, 109.5, 109.5],
    '燃煤': [964.9, 964.9, 964.9, 964.9, 964.9, 964.9, 964.9, 964.9],
    '民營電廠-燃煤': [280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0],
    '核能': [792.2, 792.2, 792.2, 792.2, 792.2, 792.2, 792.2, 792.2],
    '燃煤': [188.0, 188.0, 188.0, 188.0, 188.0, 188.0, 188.0, 188.0]
}

# 轉換為 DataFrame
df = pd.DataFrame(data)

# 創建堆積面積圖
fig = go.Figure()

for col in df.columns[1:]:
    fig.add_trace(go.Scatter(
        x=df['time'],
        y=df[col],
        mode='lines',
        stackgroup='one',
        name=col
    ))

# 設置圖表標題和標籤
fig.update_layout(
    title='今日用電曲線圖 - 依燃料類別 (Load Curve of Today - by Fuel Type)',
    xaxis_title='時間',
    yaxis_title='單位: 百萬瓦',
    yaxis=dict(type='linear'),
    legend_title='燃料類別'
)

# 顯示圖表
fig.show()


In [39]:
import plotly.graph_objs as go
import pandas as pd

# 定義數據
data = {
    'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00'],
    '太陽能': [23.7, 23.7, 23.7, 23.7, 23.7, 23.7, 23.7, 23.7],
    '其他再生能源': [2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1],
    '儲能': [0, 0, 0, 0, 0, 0, 0, 0],
    '風力': [23.5, 23.5, 23.5, 23.5, 23.5, 23.5, 23.5, 23.5],
    '水力': [20.2, 20.2, 20.2, 20.2, 20.2, 20.2, 20.2, 20.2],
    '抽蓄': [3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6],
    '燃油': [16.2, 16.2, 16.2, 16.2, 16.2, 16.2, 16.2, 16.2],
    '民營電廠-燃氣': [225.6, 225.6, 225.6, 225.6, 225.6, 225.6, 225.6, 225.6],
    '燃氣': [109.5, 109.5, 109.5, 109.5, 109.5, 109.5, 109.5, 109.5],
    '燃煤': [964.9, 964.9, 964.9, 964.9, 964.9, 964.9, 964.9, 964.9],
    '民營電廠-燃煤': [280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0],
    '核能': [792.2, 792.2, 792.2, 792.2, 792.2, 792.2, 792.2, 792.2],
    '燃煤2': [188.0, 188.0, 188.0, 188.0, 188.0, 188.0, 188.0, 188.0]
}

# 轉換為 DataFrame
df = pd.DataFrame(data)

# 創建堆積面積圖
fig = go.Figure()

for col in df.columns[1:]:
    fig.add_trace(go.Scatter(
        x=df['time'],
        y=df[col],
        mode='lines',
        stackgroup='one',
        name=col,
        hoverinfo='x+y+name'
    ))

# 設置圖表標題和標籤
fig.update_layout(
    title='今日用電曲線圖 - 依燃料類別 (Load Curve of Today - by Fuel Type)',
    xaxis_title='時間',
    yaxis_title='單位: 百萬瓦',
    yaxis=dict(type='linear'),
    legend_title='燃料類別',
    hovermode='x unified'  # 統一顯示 hover 信息
)

# 顯示圖表
fig.show()


## 電力週報案例二：頻率觸發時間分析

### 向量化與傳統for迴圈的比較

In [40]:
a = [1, 2, 3, 4, 5]
b = [1, 2, 3, 4, 5]
[x * y for x, y in zip(a, b)]


[1, 4, 9, 16, 25]

In [41]:
a = np.array([1, 2, 3, 4, 5])
b = np.array([1, 2, 3, 4, 5])
a * b

array([ 1,  4,  9, 16, 25])

In [42]:
import random
a = [random.randint(1, 10) for _ in range(1000000)]
b = [random.randint(1, 10) for _ in range(1000000)]
%timeit res = [x * y for x, y in zip(a, b)]

81.8 ms ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
a = np.random.randint(1, 10, 1000000)
b = np.random.randint(1, 10, 1000000)
%timeit a * b

2.23 ms ± 270 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 頻率分析

In [44]:
# Local Site only
url = 'https://raw.githubusercontent.com/godspeed833/course_demo_files/master/tpc_freq_colab.csv'
df_tpc = pd.read_csv(url)


In [45]:
df_tpc.timestamp = pd.to_datetime(df_tpc.timestamp)

In [46]:
df_tpc

,timestamp,tpc
0,2023-07-01 00:00:00,59.97
1,2023-07-01 00:00:01,59.96
2,2023-07-01 00:00:02,59.96
3,2023-07-01 00:00:03,59.96
4,2023-07-01 00:00:04,59.96
...,...,...
345595,2023-07-04 23:59:55,60.00
345596,2023-07-04 23:59:56,59.99
345597,2023-07-04 23:59:57,59.99
345598,2023-07-04 23:59:58,59.99


In [47]:
df_tpc.shape

(345600, 2)

In [48]:
# def freq_sum(value):
#     return value.count()/10627200
# df_tpc.pivot_table('tpc', [], bin_freq, aggfunc=[freq_sum] )

In [49]:
bin_freq = pd.cut(df_tpc['tpc'], [59, 59.75, 59.86, 59.98, 60, 60.02, 60.14, 60.25, 61 ])
df_tpc.pivot_table('tpc', df_tpc.timestamp.dt.month, bin_freq, aggfunc=['count'] )


count                                              \
tpc       (59.0, 59.75] (59.75, 59.86] (59.86, 59.98] (59.98, 60.0]   
timestamp                                                             
7                     0            230         153054         44021   

                                                                     
tpc       (60.0, 60.02] (60.02, 60.14] (60.14, 60.25] (60.25, 61.0]  
timestamp                                                            
7                 47132         100845            318             0

In [50]:
pd.cut(df_tpc['tpc'], [59, 59.75, 59.86, 59.98, 60, 60.02, 60.14, 60.25, 61 ])

0         (59.86, 59.98]
1         (59.86, 59.98]
2         (59.86, 59.98]
3         (59.86, 59.98]
4         (59.86, 59.98]
               ...      
345595     (59.98, 60.0]
345596     (59.98, 60.0]
345597     (59.98, 60.0]
345598     (59.98, 60.0]
345599    (59.86, 59.98]
Name: tpc, Length: 345600, dtype: category
Categories (8, interval[float64, right]): [(59.0, 59.75] < (59.75, 59.86] < (59.86, 59.98] <
                                           (59.98, 60.0] < (60.0, 60.02] < (60.02, 60.14] <
                                           (60.14, 60.25] < (60.25, 61.0]]

In [51]:
def get_sReg_fast(freq):
    sreg_trigger = np.where(freq <= 59.88, 1, np.nan)
    sreg_trigger = np.where(freq >= 59.98, 0, sreg_trigger)
    if pd.isna(sreg_trigger[0]):    # 給起始一個值
        sreg_trigger[0] = 0
    sreg_trigger = pd.Series(sreg_trigger).fillna(method="ffill")

    return(sreg_trigger)

In [52]:
def get_sReg(freq):
    sreg_trigger = pd.Series([0]*len(freq))
    flag = False
    for i, value in enumerate(freq):
        if value <= 59.88: #觸發條件
            flag = True
        elif (value >= 59.98): #結束條件
            flag = False
        if flag:
            sreg_trigger[i] = 1

    return(sreg_trigger)

In [53]:
# 只比前100天的頻率資料
df_tpc2 = df_tpc.iloc[:86400*100, :].copy()

In [54]:
# 1. 加速版本
a = get_sReg_fast(df_tpc2.tpc)

In [55]:
# 2. 原始版本
b = get_sReg(df_tpc2.tpc)

In [56]:
(a == b).sum()

345600

In [57]:
get_sReg_fast(df_tpc2.tpc).value_counts()

0.0    341422
1.0      4178
Name: count, dtype: int64

In [58]:
get_sReg(df_tpc2.tpc).value_counts()

0    341422
1      4178
Name: count, dtype: int64

In [59]:
def get_sReg_chatgpt(freq):
    sreg_trigger = (freq <= 59.88).astype(int)
    flag_mask = (freq >= 59.98) & sreg_trigger.astype(bool)
    sreg_trigger[flag_mask.shift().fillna(False)] = 0
    return sreg_trigger


In [60]:
get_sReg_chatgpt(df_tpc2.tpc).value_counts()

tpc
0    344818
1       782
Name: count, dtype: int64

### 將標記完資料，統計持續時間及執行次數

In [62]:
!pip install pyrle -q

In [63]:
from pyrle import Rle

In [64]:
# 計算1連續的長度
def gen_rle_length(series):
    a = Rle(series)
    sReg_sec = [x for x, y in zip(a.runs, a.values) if y == 1]
    return sReg_sec

# 產生一個array1的value的起始或結束位置的index
def gen_endpoint_index(array1, isStart, value):
    if isStart == True:
        index1 = [i for i in range(len(array1)) if array1[i] == value and (i == 0 or array1[i - 1] != value)]
    else:
        index1 = [i for i in range(len(array1)) if array1[i] == value and (i == (len(array1)-1) or array1[i + 1] != value)]

    return index1


In [65]:
df_tpc2['sreg_triggered'] =  get_sReg_fast(df_tpc2.tpc)

In [66]:
df_tpc2.sreg_triggered.value_counts()

sreg_triggered
0.0    341422
1.0      4178
Name: count, dtype: int64

In [67]:
df_tpc2.head()

,timestamp,tpc,sreg_triggered
0,2023-07-01 00:00:00,59.97,0.0
1,2023-07-01 00:00:01,59.96,0.0
2,2023-07-01 00:00:02,59.96,0.0
3,2023-07-01 00:00:03,59.96,0.0
4,2023-07-01 00:00:04,59.96,0.0


In [68]:
sreg_len = gen_rle_length(df_tpc2.sreg_triggered)
start_idx_lst = gen_endpoint_index(df_tpc2.sreg_triggered, True, 1.0)


In [69]:
df_result = df_tpc2.loc[start_idx_lst].copy()
df_result['time_len'] = sreg_len
#df_result[['timestamp', 'time_len']]

In [70]:
df_result

,timestamp,tpc,sreg_triggered,time_len
6461,2023-07-01 01:47:41,59.88,1.0,49
7443,2023-07-01 02:04:03,59.88,1.0,56
7973,2023-07-01 02:12:53,59.88,1.0,108
9909,2023-07-01 02:45:09,59.88,1.0,234
35902,2023-07-01 09:58:22,59.88,1.0,122
36104,2023-07-01 10:01:44,59.88,1.0,344
37083,2023-07-01 10:18:03,59.88,1.0,203
37427,2023-07-01 10:23:47,59.88,1.0,1110
46065,2023-07-01 12:47:45,59.88,1.0,334
46788,2023-07-01 12:59:48,59.88,1.0,858


In [71]:
df_result.groupby(df_result.timestamp.dt.to_period('W')).agg({'sreg_triggered':'count', 'time_len':'mean'}).round()

,sreg_triggered,time_len
timestamp,,
2023-06-26/2023-07-02,15,240.0
2023-07-03/2023-07-09,6,96.0


In [72]:
df_result.info()


<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 6461 to 303695
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   timestamp       21 non-null     datetime64[ns]
 1   tpc             21 non-null     float64       
 2   sreg_triggered  21 non-null     float64       
 3   time_len        21 non-null     int64         
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 840.0 bytes


In [73]:
df_result.timestamp.dt.to_period('W')

6461      2023-06-26/2023-07-02
7443      2023-06-26/2023-07-02
7973      2023-06-26/2023-07-02
9909      2023-06-26/2023-07-02
35902     2023-06-26/2023-07-02
36104     2023-06-26/2023-07-02
37083     2023-06-26/2023-07-02
37427     2023-06-26/2023-07-02
46065     2023-06-26/2023-07-02
46788     2023-06-26/2023-07-02
69257     2023-06-26/2023-07-02
69522     2023-06-26/2023-07-02
70608     2023-06-26/2023-07-02
70771     2023-06-26/2023-07-02
120752    2023-06-26/2023-07-02
191015    2023-07-03/2023-07-09
219453    2023-07-03/2023-07-09
219786    2023-07-03/2023-07-09
264080    2023-07-03/2023-07-09
287885    2023-07-03/2023-07-09
303695    2023-07-03/2023-07-09
Name: timestamp, dtype: period[W-SUN]

## 電力週報案例三 用電大戶分析

In [79]:
type = 4
# 高壓三段式
if type == 3:
    df_tou = pd.read_csv('https://raw.githubusercontent.com/godspeed833/course_demo_files/master/tou_price_%E9%AB%98%E5%A3%93%E4%B8%89%E6%AE%B5%E5%BC%8F_2023.csv')
# 高壓批次生產
elif type == 4:
    df_tou = pd.read_csv('https://raw.githubusercontent.com/godspeed833/course_demo_files/master/tou_price_%E9%AB%98%E5%A3%93%E6%89%B9%E6%AC%A1_2023.csv')

df_tou.datetime = pd.to_datetime(df_tou.datetime)
df_tou

,datetime,tou,tou_tag
0,2023-01-01 00:00:00,2.16,非夏月_離峰
1,2023-01-01 00:30:00,2.16,非夏月_離峰
2,2023-01-01 01:00:00,2.16,非夏月_離峰
3,2023-01-01 01:30:00,2.16,非夏月_離峰
4,2023-01-01 02:00:00,2.16,非夏月_離峰
...,...,...,...
17515,2023-12-31 21:30:00,2.16,非夏月_離峰
17516,2023-12-31 22:00:00,2.16,非夏月_離峰
17517,2023-12-31 22:30:00,2.16,非夏月_離峰
17518,2023-12-31 23:00:00,2.16,非夏月_離峰


In [80]:
# 平均電價
df_tou.tou.mean()

3.400438356164383

In [81]:
df_tou2 = df_tou.groupby(df_tou.datetime.dt.hour).tou_tag.value_counts().reset_index(name='count')
df_tou2

,datetime,tou_tag,count
0,0,非夏月_離峰,424
1,0,夏月_離峰,306
2,1,非夏月_離峰,424
3,1,夏月_離峰,306
4,2,非夏月_離峰,424
...,...,...,...
71,21,夏月_週六半尖峰,22
72,22,非夏月_離峰,424
73,22,夏月_離峰,306
74,23,非夏月_離峰,424


In [82]:
tou_price = df_tou.groupby(df_tou.datetime.dt.hour).tou.mean()

In [83]:


# use stack bar plot to show the distribution of tou of each hour
fig = go.Figure()

# 修正stack bar plot的次序，非夏月_離峰,非夏月_週六半尖峰, 非夏月_尖峰,夏月_離峰,夏月_半尖峰, 夏月_週六半尖峰,夏月_尖峰
if type == 3:
# 三段式
    tou_tag_lst = ['非夏月_離峰', '非夏月_週六半尖峰', '非夏月_尖峰', '夏月_離峰', '夏月_週六半尖峰', '夏月_半尖峰' , '夏月_尖峰']
elif type == 4:
# 批次
    tou_tag_lst = ['非夏月_離峰', '非夏月_週六半尖峰', '非夏月_尖峰', '夏月_離峰', '夏月_週六半尖峰', '夏月_尖峰']

df_tou2['tou_tag'] = pd.Categorical(df_tou2['tou_tag'], categories=tou_tag_lst, ordered=True)

df_tou2 = df_tou2.sort_values(by=['datetime', 'tou_tag'])
df_tou2['tou_tag'] = df_tou2['tou_tag'].astype(str)

# fig = go.Figure()
for tou_tag in tou_tag_lst:
    # 選擇 非夏月_離峰,非夏月_週六半尖峰, 非夏月_尖峰,夏月_離峰,夏月_半尖峰, 夏月_週六半尖峰,夏月_尖峰的顏色，
    # 給個紅色漸層，由淺到深，夏月_尖峰最深

    # 三段式
    if type == 3:
        if tou_tag == '非夏月_離峰':
            color = 'rgb(255, 255, 255)'
        elif tou_tag == '非夏月_週六半尖峰':
            color = 'rgb(255, 214, 214)'
        elif tou_tag == '非夏月_尖峰':
            color = 'rgb(255, 163, 163)'
        elif tou_tag == '夏月_離峰':
            color = 'rgb(255, 112, 112)'
        elif tou_tag == '夏月_週六半尖峰':
            color = 'rgb(255, 61, 61)'
        elif tou_tag == '夏月_半尖峰':
            color = 'rgb(200, 0, 0)'
        elif tou_tag == '夏月_尖峰':
            color = 'rgb(139, 0, 0)'
    elif type == 4:
        # 批次
        if tou_tag == '非夏月_離峰':
            color = 'rgb(255, 255, 255)'
        elif tou_tag == '非夏月_週六半尖峰':
            color = 'rgb(255, 214, 214)'
        elif tou_tag == '非夏月_尖峰':
            color = 'rgb(255, 163, 163)'
        elif tou_tag == '夏月_離峰':
            color = 'rgb(255, 112, 112)'
        elif tou_tag == '夏月_週六半尖峰':
            color = 'rgb(200, 0, 0)'
        elif tou_tag == '夏月_尖峰':
            color = 'rgb(139, 0, 0)'

    # Add a tou_price line on the plot, 使用不同的y軸
    fig.add_trace(go.Bar(x=df_tou2[df_tou2.tou_tag == tou_tag].datetime, y=df_tou2[df_tou2.tou_tag == tou_tag]['count'], name=tou_tag, marker_color=color))

# tou_price line使用亮一點的藍色跟粗一點的線
fig.add_trace(go.Scatter( y=tou_price, name='tou_price', yaxis='y2', line=dict(color='blue', width=2)))
# fig.add_trace(go.Scatter( y=tou_price, name='tou_price', yaxis='y2', line=dict(color='blue', width=1)))
# lengend 的位置放在下方，只保留barplot的lengend，tou_price的lengend不顯示
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

if type == 3:
    fig.update_layout(barmode='stack', xaxis_title='Hour', yaxis_title='小時數', title='2023年的高壓三段式時間電價分佈圖')
elif type == 4:
    fig.update_layout(barmode='stack', xaxis_title='Hour', yaxis_title='0.5小時數', title='2023年的高壓批次生產時間電價分佈圖')


fig.update_layout(yaxis2=dict(title='小時平均價格', overlaying='y', side='right'))



fig.show()

    # fig.add_trace(go.Bar(x=df_tou2[df_tou2.tou_tag == tou_tag].datetime, y=df_tou2[df_tou2.tou_tag == tou_tag]['count'], name=tou_tag, marker_color=color))




# 到此結束，謝謝大家